In [ ]:
import numpy as np
import scipy
import pandas as pd

In [ ]:
# Install optimization library
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
from google.colab import drive

drive.mount('/content/drive') # This will ask for permisions.

# Point to the folder where you have the results of the Form.
!ls "/content/drive/My Drive/AreYouTheOne/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 AreYouTheOne.ipynb		'Game Groups and Couples.docx'	 responses.csv
'Forms Responses Example.xlsx'	'Groups Grid.xlsx'


# Format of the Test!

### First question: Name and Surname
### Second question: Are you coming to the activity?
In the majority of the groups, people that is not coming will want to answer the test. With this question, you can filter them out to run the code and extract the pairs for the game with just the people that are coming.

Later on, you can run the code with everyone to extract the full compatibilties and the "global" pairs.

## Test questions
In this code, you do not care about the question itself, you will just parse the last letter of the question (A, B, or C). This is the type of question.
- A: Points are given only if the answer is exactly the same.
- B: Points are given if the answer is complementary (which side of the bed do you sleep on?)
- C: Points are given depending on the proximity of the answer.

## Test answers
In this code, you do not care about the answer itself, you will just parse the first number of the answer.

***That means, be very careful on puting letters at the end of every question of the test, and numbers at the begining of the answers, this will make your life MUCH easy.***

In [ ]:
# IMPORTANT! I don't know how to read the drive excel format.
# So I just download it as csv and upload it again. You can probably optimize this.
df = pd.read_csv("/content/drive/My Drive/AreYouTheOne/responses.csv")

# Keep only the people that are coming
df = df[df[df.columns[2]] == '1. Yes, of course']

# If the number of people is odd, you'll have to remove yourself from the pairings, I'm sorry ^^
df = df[df['Name and surname'] != 'Marcel']

# Update indexing of the table after removing people
df = df.reset_index(drop=True)

In [ ]:
# This is what happens if you forget to put numbers in the answers of the test. You remove the answers.
df = df.drop('How many hours of sleep do you need to function? (B)', axis=1)
df = df.drop('How many cups of coffee do you drink daily? (B)', axis=1)

In [ ]:
# Nom = Name in catalan ^^
nomStr = 'Name and surname'
noms = df[nomStr].values

In [ ]:
for nom in noms:
  print(nom)

Dani Marín
Subhadeep Sarkar
Emanuele Fondi
Andrija Zupic
Laia Montellà
Lidia Lozano Martin
Chloé Padois
Sagar Malhotra (last year's most compatible person)
Marie Schölch
Marine Pihet
Mar Carretero-Castrillo
Nooshineh
Pablo Tejerina
Adriana Nadal
Anya Paopiamsap
Sandra Tomas
Judith Ardèvol
marisol traforetti
Sergi Novell
Toni Soler Terricabras
Julien Poyatos
Helena U
Stefano Trezzi
Guillermo Martínez-Somonte
Lucía Castells
Ingrid Beloto


In [ ]:
# Evaluate compatibility of answers pa and pb in a question of type t.
# If type == B, normalize the difference by b_max, the number of questions in the range.
# b_max is set up manualy in the next function (sorry, hardcoded hehehe)
def preg_eval(pa, pb, t, b_max=1):

  if t == 'A':
    if pa == pb:
      return 1
    else:
      return 0
  if t == 'B':
    #print(pa, pb)
    val = np.abs(int(pa) - int(pb))
    return 1-(val/b_max)
  if t == 'C':
    if pa == pb:
      return 0
    else:
      return 1
  return 0

# Evaluate full compatibility of two persons.
def rows_compat(ra, rb):

  index = ra.index
  index_type = []

  points = 0

  for i, preg in enumerate(index):
    if i<3 or i == 13:
      index_type.append('0')
      continue
    found = False
    pos = -1
    while not found:
      #print(pos, preg[pos])
      if preg[pos] in ['A', 'B','C']:
        found = True
      else:
        pos -=1

    pa = str(ra[preg])[0]
    pb = str(rb[preg])[0]

    b_val = 1
    # Here we hardcode the range (number of answers) in the B-type questions.
    if i ==14 or i == 15 or i == 16:
      b_val = 5
    if i == 17:
      b_val = 7
    #print(i, preg, b_val)
    points += preg_eval(pa, pb, preg[pos], b_max = b_val)

  return points
  #print(ra, rb)

# Populate compatibility matrix A. Keep the diagonal (compatibility to yourself) = 0.
n = len(df)
A = np.zeros((n,n))
for index_i, row_i in df.iterrows():
  for index_j, row_j in df.iterrows():
    if index_i == index_j:
      A[index_i, index_j] = 0
      continue
    #print(index_i, index_j)
    A[index_i, index_j] = rows_compat(row_i, row_j)
    #break
  #break

From input: $A_{i,j} = $ match punctuation between persons $i$ and $j$.

Variables: $x_{i,j} = 1$ if the pair is selected, $0$ if the pair is not selected.

Goal: Maximize the total compatibility.

Restrictions:
- (1) Each person $i$, can only be in one pair in total.
- (2) If person $i$ is in a pair with person $j$, person $j$ is in a pair with person $i$.
- (3) $x_{i,j}$ can only be $0$ or $1$.

## Mathematical expressions

\begin{equation}
\begin{split}
\text{maximize} \displaystyle\sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} A_{i,j}&x_{i,j} \\
\text{subject to} &\displaystyle\sum\limits_{j=1}^{n} x_{i,j} = 1,\quad  \forall i=1 ,..., n\\
&x_{i,j} = x_{j,i}, \quad \quad \forall i=1 ,..., n \quad \forall j=1 ,..., m \\
&x_{i,j} \in \{0,1\}, \quad \forall i=1 ,..., n \quad \forall j=1 ,..., m
\end{split}
\end{equation}

In [ ]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

x = {}
for i in range(n):
  for j in range(n):
      x[i,j] = solver.IntVar(0, 1, "x[{},{}]".format(i,j))

print("Number of variables =", solver.NumVariables())

for i in range(n):
  solver.Add(sum(x[i,j] for j in range(n)) == 1)

#for j in range(n):
#  solver.Add(sum(x[i,j] for i in range(n)) == 1)

for i in range(n):
  for j in range(i+1, n):
    solver.Add(x[i,j] == x[j,i])

#solver.Add(sum(x[i,j] for i in range(n) for j in range(n)) == n)

Number of variables = 676


In [ ]:
objective = solver.Objective()

for i in range(n):
  for j in range(n):
    objective.SetCoefficient(x[i,j], A[i,j])

In [ ]:
objective.SetMaximization()


In [ ]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with SCIP 9.2.0 [LP solver: Glop 9.12]


In [ ]:
pairs = []

if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())
    A_res = np.zeros_like(A)

    it = 0
    for i in range(n):
      for j in range(n):
            #print(x[i,j].name(), " = ", x[i,j].solution_value())
            A_res[i,j] = x[i,j].solution_value()

            if j>i and x[i,j].solution_value()>0.5:

              pairs.append([noms[i], noms[j]])

    #print(A_res)
    print()
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    print(f"Problem solved in {solver.nodes():d} branch-and-bound nodes")
else:
    print("The problem does not have an optimal solution.")

Objective value = 425.9428571428571

Problem solved in 4632 milliseconds
Problem solved in 56 iterations
Problem solved in 1 branch-and-bound nodes


Now we have the pairs organized. To do the game, I usually split the pairs in diferent groups. Here we evaluate the compatibility within a group, to try to also maximize it (while maintaining the pairs fixed).

To do so, I shuffle the pairs, evaluate the compatibility of the groups and do the process 1000000 times to maximize the compatibility.

In [ ]:
# Evaluate compatibility within a group.
def evalGroup(group):

  totalsum = 0
  for personA in group:
    i = np.argwhere(personA == noms)[0][0]
    for personB in group:
      j = np.argwhere(personB == noms)[0][0]

      if i<=j:
        continue
      totalsum += A[i,j]

  return totalsum


In [ ]:
import random

groupFinA = []
groupFinB = []
groupFinC = []

groupFinApairs = []
groupFinBpairs = []
groupFinCpairs = []

maxComp = -1

for i in range(100):


  random.shuffle(pairs)

  groupA = []
  groupB = []
  groupC = []

  groupApairs = []
  groupBpairs = []
  groupCpairs = []

  for i, pair in enumerate(pairs):

    # Hardcode these ifs if you need more groups or different sizes.
    if i<4: # Size of group A
      groupA.append(pair[0])
      groupA.append(pair[1])
      groupApairs.append(pair)
    elif i < 8: # Size of group B
      groupB.append(pair[0])
      groupB.append(pair[1])
      groupBpairs.append(pair)
    else: # Size of group C
      groupC.append(pair[0])
      groupC.append(pair[1])
      groupCpairs.append(pair)

  comp = evalGroup(groupA) + evalGroup(groupB) + evalGroup(groupC)

  if comp>maxComp:
    maxComp = comp
    groupFinA = groupA
    groupFinB = groupB
    groupFinC = groupC

    groupFinApairs = groupApairs
    groupFinBpairs = groupBpairs
    groupFinCpairs = groupCpairs

In [ ]:
print('Final groups and pairs')
print()
print('A')
for pair in groupFinApairs:
  print(pair[0],'-',pair[1])

print('B')
for pair in groupFinBpairs:
  print(pair[0],'-',pair[1])

print('C')
for pair in groupFinCpairs:
  print(pair[0],'-',pair[1])

print()


print('Final groups shuffled to hide the pairs.')
print()

random.shuffle(groupFinA)
random.shuffle(groupFinB)
random.shuffle(groupFinC)
print('A')
for person in groupFinA:
  print(person)

print('B')
for person in groupFinB:
  print(person)

print('C')
for person in groupFinC:
  print(person)

Final groups and pairs

A
Marie Schölch - marisol traforetti
Laia Montellà - Ingrid Beloto
Nooshineh - Lucía Castells
Sagar Malhotra (last year's most compatible person) - Julien Poyatos
B
Sandra Tomas - Guillermo Martínez-Somonte
Judith Ardèvol - Helena U
Lidia Lozano Martin - Sergi Novell
Marine Pihet - Anya Paopiamsap
C
Andrija Zupic - Chloé Padois
Emanuele Fondi - Pablo Tejerina
Dani Marín - Adriana Nadal
Mar Carretero-Castrillo - Toni Soler Terricabras
Subhadeep Sarkar - Stefano Trezzi

Final groups shuffled to hide the pairs.

A
Nooshineh
Sagar Malhotra (last year's most compatible person)
marisol traforetti
Ingrid Beloto
Julien Poyatos
Laia Montellà
Lucía Castells
Marie Schölch
B
Helena U
Marine Pihet
Lidia Lozano Martin
Anya Paopiamsap
Judith Ardèvol
Guillermo Martínez-Somonte
Sergi Novell
Sandra Tomas
C
Toni Soler Terricabras
Chloé Padois
Stefano Trezzi
Andrija Zupic
Adriana Nadal
Subhadeep Sarkar
Pablo Tejerina
Dani Marín
Emanuele Fondi
Mar Carretero-Castrillo


In [ ]:
# For each person, print the two top compatibilities and the bottom two.
# Ready to copy and send as a file after the game.

for i, nom_i in enumerate(noms):

  comp = A[i,:]
  idx_comp = np.argsort(comp)
  #print(nom_i, 'min',noms[idx_comp[1:3]],comp[idx_comp[1:3]],'max',noms[idx_comp[-2:]],comp[idx_comp[-2:]])
  print(nom_i)
  str_row = 'Min: '
  for it, idx in enumerate(idx_comp[1:3]):
    str_row += noms[idx] + ' (' +"{:.1f}".format(comp[idx]) + ')'
    if it == 0:
      str_row += ', '
  str_row += '.'
  print(str_row)
  str_row = 'Max: '
  for it, idx in enumerate(idx_comp[-2:]):
    str_row += noms[idx] + ' (' +"{:.1f}".format(comp[idx]) + ')'
    if it == 0:
      str_row += ', '
  print(str_row + '.')
  print()

Dani Marín
Min: Sandra Tomas (7.3), Sergi Novell (9.0).
Max: Adriana Nadal (16.6), Subhadeep Sarkar (17.7).

Subhadeep Sarkar
Min: Mar Carretero-Castrillo (7.7), Sandra Tomas (8.8).
Max: Andrija Zupic (16.2), Dani Marín (17.7).

Emanuele Fondi
Min: Helena U (8.4), Ingrid Beloto (8.5).
Max: Dani Marín (16.5), Pablo Tejerina (18.7).

Andrija Zupic
Min: Julien Poyatos (9.6), Sergi Novell (10.0).
Max: Stefano Trezzi (16.5), Chloé Padois (16.6).

Laia Montellà
Min: Helena U (6.7), Pablo Tejerina (9.6).
Max: Ingrid Beloto (16.2), Mar Carretero-Castrillo (16.4).

Lidia Lozano Martin
Min: Ingrid Beloto (7.8), Nooshineh (7.8).
Max: Sandra Tomas (13.3), Helena U (15.1).

Chloé Padois
Min: Sergi Novell (6.6), Marie Schölch (9.1).
Max: Anya Paopiamsap (15.6), Andrija Zupic (16.6).

Sagar Malhotra (last year's most compatible person)
Min: Lucía Castells (7.2), Sandra Tomas (7.3).
Max: Pablo Tejerina (14.8), Julien Poyatos (15.5).

Marie Schölch
Min: Subhadeep Sarkar (9.0), Lidia Lozano Martin (9.1)

In [ ]:
# Store all the compatibilities in a file. Then with a bit of w

np.savetxt("/content/drive/My Drive/AreYouTheOne/compatibility.csv", A, delimiter=",")